<a href="https://colab.research.google.com/github/y0o0ic/AIS_Visualization/blob/master/AIS_big_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# インポート

In [0]:
import os
import glob
import pandas as pd
import numpy as np
import plotly.express as px
import datetime as dt

# データのダウンロード　（1年の全体容量97GB：半年分をダウンロード）

In [0]:
# ! wget -np -r -nH -L --cut-dirs=3 https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2018/

In [0]:
# dirs = os.listdir("2018/")

In [6]:
# dirs[0:10]

['AIS_2018_03_05.zip',
 'AIS_2018_03_11.zip',
 'AIS_2018_04_18.zip',
 'AIS_2018_04_30.zip',
 'AIS_2018_04_24.zip',
 'AIS_2018_06_09.zip',
 'AIS_2018_06_20.zip',
 'AIS_2018_06_08.zip',
 'AIS_2018_04_25.zip',
 'AIS_2018_03_10.zip']

In [0]:
! unzip "2018/AIS_2018_0*.zip"

Archive:  2018/AIS_2018_03_05.zip
  inflating: AIS_2018_03_05.csv      

Archive:  2018/AIS_2018_03_11.zip
  inflating: AIS_2018_03_11.csv      

Archive:  2018/AIS_2018_04_18.zip
  inflating: AIS_2018_04_18.csv      

Archive:  2018/AIS_2018_04_30.zip
  inflating: AIS_2018_04_30.csv      

Archive:  2018/AIS_2018_04_24.zip
  inflating: AIS_2018_04_24.csv      

Archive:  2018/AIS_2018_06_09.zip
  inflating: AIS_2018_06_09.csv      

Archive:  2018/AIS_2018_06_20.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
Archive:  2018/AIS_2018_06_08.zip
  inflating: AIS_2018_06_08.csv      

Archive:  2018/AIS_2018_04_25.zip
  inflating: AIS_2018_04_25.csv      

Archive:  2018/AIS_2018_03_10.zip
  inflating: AIS_2018_03_10.csv      

Archive:  2018/AIS_2018_04_19.zip
  inflating: AIS_201

In [0]:
! rm -r 2018/AIS_2018_01_*.zip

In [0]:
csvs = glob.glob("*.csv")

In [0]:
csvs.sort()

In [0]:
# rdf = pd.read_csv(csvs[0])

In [0]:
# rdf['datetime'] = pd.to_datetime(rdf['BaseDateTime'])
# rdf.dropna(subset=['VesselType'],inplace=True)
# rdf["iVesselType"]=rdf["VesselType"].astype(np.int64)
# rdf.query('VesselType in [31]',inplace=True)
# # rdf.query('VesselType in @iships',inplace=True)

In [0]:
# rdf.head()

In [0]:
# ? rdf.dropna

# Ship Type についてまとめてみた

In [0]:
ShipType=pd.read_csv("AIS_VesselType.csv",index_col=0)

In [0]:
# ShipType.columns

In [0]:
# ShipType["Description"].unique()

## 2種類の書き方あり
### １つ目

In [0]:
# SelectShip=ShipType[ShipType["Description"].str.contains("Passenger")]

### ２つ目

In [0]:
SelectType=['Military ops','Search and Rescue vessel',
            'Passenger, all ships of this type',
       'Passenger, Hazardous category A',
       'Passenger, Hazardous category B',
       'Passenger, Hazardous category C',
       'Passenger, Hazardous category D',
       'Passenger, Reserved for future use',
       'Passenger, No additional information']
SelectShip=ShipType.query('Description in @SelectType')

In [0]:
iships=SelectShip.index

In [0]:
# iships

In [0]:
# SelectShip

# 限定したShip Typeのデータを抽出します。

In [0]:
? pd.read_csv

In [0]:
# idf=pd.read_csv(csvs[0])

In [0]:
# idf.head()

In [0]:
# len(idf['VesselName'].unique())

In [0]:
# iidf=idf[~idf.duplicated(subset=['VesselName'],keep='first')]

In [0]:
# len(iidf)

In [0]:
dlist=[]
for fi in csvs[]:
  rdf = pd.read_csv(fi,usecols=[1,2,3,4,5,6,7,8,10,11,12,13,14,15])
  # rdf=rdf.dropna(subset=["VesselName"])
  # rdf.dropna(subset=['VesselType'],inplace=True)
  rdf.query('VesselType in @iships',inplace=True)
  rdf=rdf[~rdf.duplicated(subset=['VesselName'],keep='first')]
  dlist.append(rdf)

In [0]:
df_AIS = pd.concat(dlist, axis=0, ignore_index=True)

In [0]:
df_AIS["VesselType"]=df_AIS["VesselType"].astype(np.int64)
df_AIS['VesselType']=df_AIS["VesselType"].replace(SelectShip.index, SelectShip["Description"])

In [0]:
df_AIS["date"]=pd.to_datetime(df_AIS['BaseDateTime']).dt.round(freq = 'D')

In [0]:
df_AIS.to_pickle("df_AIS")

In [0]:
df_AIS=pd.read_pickle("df_AIS")

In [291]:
df_AIS.head(3)

,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,VesselType,Status,Length,Width,Draft,Cargo,date
0,2018-01-01T00:00:01,37.98799,-122.41855,37.0,-189.4,223.0,SOLANO,IMO8988416,"Passenger, all ships of this type",0.0,41.0,11.0,1.5,NaN,2018-01-01
1,2018-01-01T00:00:02,38.62288,-90.18316,0.0,-99.1,511.0,TOM SAWYER,NaN,"Passenger, all ships of this type",15.0,25.0,NaN,NaN,NaN,2018-01-01
2,2018-01-01T00:00:05,30.39578,-81.43012,0.3,-118.4,117.0,JEAN RIBAULT,NaN,"Passenger, all ships of this type",0.0,46.0,17.0,NaN,NaN,2018-01-01


In [292]:
df_AIS.columns

Index(['BaseDateTime', 'LAT', 'LON', 'SOG', 'COG', 'Heading', 'VesselName',
       'IMO', 'VesselType', 'Status', 'Length', 'Width', 'Draft', 'Cargo',
       'date'],
      dtype='object')

In [293]:
print(len(df_AIS))
df_AIS.describe()

4750


,LAT,LON,SOG,COG,Heading,Status,Length,Width,Draft,Cargo
count,4750.000000,4750.000000,4750.000000,4750.000000,4750.000000,3617.000000,4291.000000,3933.000000,1260.000000,1109.000000
mean,35.602942,-95.947863,3.321347,-20.444737,418.135368,3.659663,54.874155,11.277905,4.145159,53.868350
std,8.959663,21.673658,6.573117,112.350635,156.546236,5.591408,68.203210,8.646341,2.411542,18.534162
min,15.712670,-159.355130,-0.100000,-204.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,29.256158,-119.531252,0.000000,-110.300000,325.000000,0.000000,20.000000,6.000000,2.300000,51.000000
50%,33.763050,-90.183190,0.000000,-49.600000,511.000000,0.000000,30.000000,8.000000,3.200000,60.000000
75%,41.172780,-77.471682,1.700000,73.850000,511.000000,5.000000,52.000000,12.000000,5.600000,60.000000
max,70.508500,-62.593350,37.000000,204.700000,511.000000,15.000000,362.000000,60.000000,10.300000,99.000000


In [0]:
ships=(df_AIS[df_AIS.duplicated(subset=["VesselName"],keep="first")]).loc[:,['VesselName',
       'IMO', 'VesselType', 'Status', 'Length', 'Width', 'Draft', 'Cargo']]

In [295]:
ships.head(3)

,VesselName,IMO,VesselType,Status,Length,Width,Draft,Cargo
1002,H208,NaN,"Passenger, all ships of this type",0.0,NaN,NaN,NaN,NaN
1003,CRYSTAL COAST LADY,IMO8967632,"Passenger, all ships of this type",15.0,24.0,7.0,NaN,NaN
1004,MISS EFFIE,NaN,"Passenger, all ships of this type",0.0,13.0,4.0,NaN,NaN


In [0]:
ships.to_pickle('ships')

In [0]:
ships=pd.read_pickle('ships')

In [298]:
print(len(ships)/len(df_AIS))

0.7338947368421053


In [299]:
df_AIS.columns

Index(['BaseDateTime', 'LAT', 'LON', 'SOG', 'COG', 'Heading', 'VesselName',
       'IMO', 'VesselType', 'Status', 'Length', 'Width', 'Draft', 'Cargo',
       'date'],
      dtype='object')

In [0]:
df=df_AIS.loc[:,['VesselName','IMO', 'VesselType', 'Length','date','LAT', 'LON', 'SOG', 'COG', 'Heading',]]

In [302]:
df.head(2)

,VesselName,IMO,VesselType,Length,date,LAT,LON,SOG,COG,Heading
0,SOLANO,IMO8988416,"Passenger, all ships of this type",41.0,2018-01-01,37.98799,-122.41855,37.0,-189.4,223.0
1,TOM SAWYER,NaN,"Passenger, all ships of this type",25.0,2018-01-01,38.62288,-90.18316,0.0,-99.1,511.0


In [0]:
df.to_pickle("data")

In [0]:
df=pd.read_pickle("data")

In [307]:
len(df[df["date"]<dt.datetime(2018,1,2)])

907

In [308]:
len(df["VesselName"].unique())

1264

In [313]:
fig=px.scatter_geo(df,
                     lat="LAT", 
                     lon="LON",
                     color="VesselType",
                     hover_name="VesselName",
                     hover_data=["VesselName","Length"])
fig.show()

In [310]:
fig = px.scatter_mapbox(df[df["date"]<dt.datetime(2018,1,2)],
                     lat="LAT", 
                     lon="LON",
                     color="VesselType",
                     hover_name="VesselName",
                     hover_data=["VesselName","Length"])
fig.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }
      ])
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [0]:
import plotly.graph_objects as go

In [0]:
steplist=df["datetime"].unique

In [0]:
steplist

<bound method Series.unique of 0         2018-01-01
1         2018-01-01
2         2018-01-01
3         2018-01-01
4         2018-01-01
             ...    
2677841   2018-01-06
2677842   2018-01-06
2677843   2018-01-06
2677844   2018-01-06
2677845   2018-01-06
Name: datetime, Length: 2677846, dtype: datetime64[ns]>

In [0]:
fig = px.scatter_geo(df,
                     lat="LAT", 
                     lon="LON",
                     color="ShipType",
                     hover_name="VesselName",
                     hover_data=["ShipType","IMO","Length"],
                     animation_frame="datetime")
fig.show()

KeyboardInterrupt: ignored